In [3]:
!pip install -q cassio datasets langchain openai tiktoken

In [4]:
!pip install PyPDF2

In [5]:
!pip install --upgrade astrapy

In [6]:
!pip install langchain_community huggingface_hub

In [7]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

#support dataset retrival with hugging face
from datasets import load_dataset

#with CassIO, the engine powering the Astra DB integration in LangChain
#you will also initialize the db connection
import cassio

from PyPDF2 import PdfReader

In [29]:
ASTRA_DB_APPLICATION_TOKEN = "YOUR_TOKEN"
ASTRA_DB_ID = "YOUR_ID"

In [28]:
import os
from dotenv import load_dotenv # use this instead of declaring your api_key inside the main script.
load_dotenv(override=True) # declaring api keys in the script can lead to serious security and financial concerns

hf_token = os.getenv("HF_TOKEN")

READ THE PDF

In [10]:
pdfreader = PdfReader('federal_budget_2025_2026.pdf')


In [11]:
from typing_extensions import Concatenate
#read raw test from pdf

raw_text = " "

for i, page in enumerate(pdfreader.pages):
  text = page.extract_text()
  if text:
    raw_text += text

In [30]:
cassio.init(token = ASTRA_DB_APPLICATION_TOKEN , database_id=ASTRA_DB_ID)

In [14]:
# Initialize embeddings
model_name = "google/gemma-3-1b-it"  #deepseek-ai/DeepSeek-R1

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [16]:
# 2. Create a Hugging Face text2text-generation pipeline
hf_pipeline = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,               # use 0 for cuda(GPU) or -1 for CPU
    #max_length=256,
    temperature=0.7,
    do_sample=True,
)

Device set to use cuda:0
The model 'Gemma3ForCausalLM' is not supported for text2text-generation. Supported models are ['PeftModelForSeq2SeqLM', 'BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'GraniteSpeechForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', '

In [17]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)

/tmp/ipython-input-17-3721297035.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [18]:
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipython-input-18-2833710984.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
Some weights of Gemma3TextModel were not initialized from the model checkpoint at google/gemma-3-1b-it and are newly initialized: ['embed_tokens.weight', 'layers.0.input_layernorm.weight', 'layers.0.mlp.down_proj.weight', 'layers.0.mlp.gate_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.0.post_attention_layernorm.weight', 'layers.0.post_feedforward_layernorm.weight', 'layers.0.pre_feedforward_layernorm.weight', 'layers.0.self_attn.k_norm.weight', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.o_pro

Create your langchain vector store backed by astra db

In [19]:
astra_vector_store =  Cassandra(embedding = embeddings,
                                table_name = "qs_mini",
                                session = None,
                                keyspace = None)

In [20]:
from langchain.text_splitter import CharacterTextSplitter
#we need to split the text using character text split, such that it should not increase the token size

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,

)
texts = text_splitter.split_text(raw_text)

In [21]:
texts[:10]

['GOVERNMENT OF P AKIS TAN\nFINANCE DIVISION\nISLAMAB ADFEDERAL BUDGE T \n2025-26PREFACE\nThe document can be accessed on Finance Division’s website\n(www.finance.gov.pk). Further details onthe contents ofthis document areavailable in\nother budget documents.\nIMDAD ULLAH BOSAL\nSecretary t o the Government of Pakistan\nFinance Division\nIslamabad, the 10th June, 2025Budget inBrief gives anoverview ofthe federal budget for FY2025-26. It\nprovides a glimpse ofkey priorities and objectives ofthe Federal Government, offers a\nsummary ofrevenue and expenditures, fiscal deficit targets and the corresponding\nfinancing plan. The document also provides a detailed breakdown oftaxand non-tax\nreceipts, projected share ofprovinces under theNFC Award, capital andexternal receipts,',
 'financing plan. The document also provides a detailed breakdown oftaxand non-tax\nreceipts, projected share ofprovinces under theNFC Award, capital andexternal receipts,\nandcurrent expenditures along with its funct

In [22]:
astra_vector_store.add_texts(texts)

print("Inserted %i headlines." % len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

Inserted 81 headlines.


In [23]:
first_question =  True

while True:
  if first_question:
    query_text = input("\nEnter your Question (or type 'quit' to exit): ").strip()

  else:
    query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

  if query_text.lower() == "quit":
    break
  if query_text == "":
    continue

  first_question = False

  print("n\QUESTION: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm = llm).strip()
  print("n\ANSWER: \"%s\"\n" % answer)

  print("FIRST DOCUMENT BY RELEVENCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print("  [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your Question (or type 'quit' to exit): Current Expenditure
n\QUESTION: "Current Expenditure"


n\ANSWER: "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Package
Restructuring of PRAL under FBR
Credit Scoring Services - SMEDA
Access to Justice Development FundProvision for SPC - SIFC
Other Information Technology Initiatives
Digitization of FBR/IT upgradation and 
restructuring etc.
USF & R&D fund
Autism Society of Pakistan
Emergency, Relief and Repatriation
FGEIs
Contd…
25Budget  Revised Budget
2024-25 2024-25 2025-26
 - -  -  8,000
 - -  220 950
 - -  -  370
 - 300 211 300
 - 50 50 100
 - -  9,000 -  
 - -  6,000 -  
 - -  292 -  
 - 250 250 -  
 - -  -  20
1,776,672 1,761,481 1,927,895Provision for Women Inclusive Finance 
(WIF)
Grant to TNF Boarding School SwabiCenter of Excellence - OPHRD
Public Facilitation Projects outside Sindh
Clearing of Liabilities of Utility 
Companies- Agencies -Partners
OthersPM Initiatives for Enhancing SMEs

Contd….FUNCTION WISE CURR

  [0.7425] "Package
Restructuring of PRAL under FBR
Credit Scoring Services - SMEDA
Access to Ju ..."
  [0.7350] "Contd….FUNCTION WISE CURRENT EXPENDITURE
Classification
 - Others(Rs in Million)TABL ..."
  [0.7194] "ratios given inclause (2) are based onmultiple indicators. The indicators and their
 ..."
  [0.7124] "Royalty on Natur al Gas
Discou nt Retained on Loc al Crude Price 
Windf all Levy aga ..."

What's your next question (or type 'quit' to exit): exit
n\QUESTION: "exit"


n\ANSWER: "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Contd….FUNCTION WISE CURRENT EXPENDITURE
Classification
 - Others(Rs in Million)TABLE - 11
 - Servicing of Domestic Debt General Public Service
 Defence Affairs and Services
 Social Protection Education Affairs and Services
 - Servicing of Foreign Debt Housing and Community Amenities
(Rs in Million) Public Order and Safety AffairsClassification
 Recreation, Culture and Religion
GENERAL PUBLIC SERVICETable-11 shows details ofCurrent Expenditure which isdivided into ten (10)
functional items as per Chart of Accounts.
 Environment Protection
 Health Affairs & Services
The details of Current Expenditure under above ten Functions are as under:
 - Superannuation Allowances & PensionsExecutive & Legislative Organs, Financial,

Package
Restructuring of PRAL under FBR
Credit Scoring Services - SMEDA
Access to Justice Devel

  [0.7135] "Contd….FUNCTION WISE CURRENT EXPENDITURE
Classification
 - Others(Rs in Million)TABL ..."
  [0.7052] "Package
Restructuring of PRAL under FBR
Credit Scoring Services - SMEDA
Access to Ju ..."
  [0.7050] "Royalty on Natur al Gas
Discou nt Retained on Loc al Crude Price 
Windf all Levy aga ..."
  [0.7047] "GREEN COMPONENT IN SUBSIDIES 
B. Indirectly Favorable
C. Neutral or not assessedFY20 ..."

What's your next question (or type 'quit' to exit): quit
